## Problema do Arpirador-de-Pó

Um robô aspirador que se encontra em uma sala quadrada dividida em n posições, tem como objetivo limpar todas as posições da sala.

### Solução utilizando Algoritmos de Busca Não-Informada (Busca Cega)

Escopo do problema:

* Estados: Posição em que o robo se encontra em um dado momento.
* Estado Inicial: Qualquer uma das posições.
* Ações Possíveis: O robo pode andar para quatro posições: esquerda, direita, cima, baixo; e pode limpar a posição.
* Modelo de Transição: A matriz da sala, mostrando onde já foi limpo.
* Teste de Objetivo: Todas as posições da sala devem estar limpas.

______________________________________________________________________________________________________________________________

- Será utilizada uma matriz de estados, correspondente ao tamanho da sala. Esta matriz pode ter N linhas e M colunas.

_Inicialmente a matriz de estados é iniciada com todas as posições = 1, equivalente a sala inteira suja. Quando uma posição é limpa, o valor da posição é modificado para 0._


In [1]:
import numpy as np
from collections import deque

linhas = 3
colunas = 3

matriz_estados = np.zeros((linhas,colunas), dtype=np.int32)

#matriz com o objetivo final, ou seja, todas as posições limpas
matriz_objetivo = matriz_estados.copy()

#matriz de estados inicial, ou seja, todas as posições sujas
matriz_estados[0:linhas, 0:colunas] = 1

custo_caminho = 1

print (matriz_estados)


[[1 1 1]
 [1 1 1]
 [1 1 1]]


In [2]:
# Método utilizado para verificar se a posição inicial definida pelo usuário é valida na matriz de estados.

def estado(linha, coluna):
    erro = True
    
    estado_inicial = [linha,coluna]

    if estado_inicial[0] > linhas or estado_inicial[1] > colunas:
        erro = False
    
    return erro

#### Ações possíveis

O robo pode realizar 5 ações, que serão armazenadas em um vetor na seguinte sequência:

- limpar
- esquerda
- direita
- cima
- baixo

Quando uma ação no vetor for igual a 1, esta ação pode ser executada, e não poderá executar quando for 0.

In [3]:
# Método utilizado para definir as ações possíveis para a posição atual do robo

def acoes_possiveis(estado):
    acoes = [1,1,1,1,1]
        
    if estado[0] == 0:
        acoes[3] = 0
        
    if estado[0] == (linhas-1):
        acoes[4] = 0
    
    if estado[1] == 0:
        acoes[1] = 0
    
    if estado[1] == (colunas-1):
        acoes[2] = 0     
    
    return acoes

In [4]:
# Método utilizado para verificar se todas as posições da matriz foram limpas

def testar_objetivo(no):
    teste_objetivo = False
    
    posicao = estado_inicial.copy()
    
    #copia da matriz original para matriz_transicao -> matriz temporária
    matriz_transicao = matriz_estados.copy()
    
    solucao = avaliar_solucao(no)
    
    for acao in solucao:
        if acao == "Limpar":       
            matriz_transicao[posicao[0]][posicao[1]] = 0
    
        elif acao == "Esquerda":
            posicao[1] -= 1

        elif acao == "Direita":
            posicao[1] += 1

        elif acao == "Cima":
            posicao[0] -= 1

        elif acao == "Baixo":
            posicao[0] += 1
            
    if np.array_equal(matriz_transicao, matriz_objetivo):
        teste_objetivo = True
        
      
    return teste_objetivo
            

In [5]:
#Método que avalia recursivamente todos os nós percorridos

def avaliar_solucao(no_final):
    vetor_solucao = []

    while not no_final.pai == None:
        vetor_solucao.append(no_final.acao)
        no_final = no_final.pai
        
    return vetor_solucao

In [6]:
class No:

    def __init__(self, estado, pai, acao, custo):
        self.estado = estado
        self.pai = pai
        self.acao = acao
        self.custo = custo
        
    def __repr__(self):
        if self.pai:
            return 'No{\n Estado: %s\n Pai: %s,\n Ação: %s,\n Custo: %s \n}' % (self.estado, self.pai.estado, self.acao, self.custo)
        else:
            return 'No{\n Estado: %s,\n Pai: %s,\n Ação: %s,\n Custo: %s \n}' % (self.estado, 'None', self.acao, self.custo)

In [11]:
def no_filho(pai, acao):
    
    estado_atual = pai.estado.copy()
       
    if acao[0] == 1:        
        acao_atual = "Limpar"
    
    elif acao[1] == 1:
        estado_atual[1] -= 1
        acao_atual = "Esquerda"
        
    elif acao[2] == 1:
        estado_atual[1] += 1
        acao_atual = "Direita"
        
    elif acao[3] == 1:
        estado_atual[0] -= 1
        acao_atual = "Cima"
    
    elif acao[4] == 1:
        estado_atual[0] += 1
        acao_atual = "Baixo"
           
    #print("possiveis %s estado pai: %s acao atual: %s estado_atual: %s" % (acoes_possiveis(pai.estado), pai.estado, acao_atual, estado_atual))
    filho = No(estado_atual, pai, acao_atual, pai.custo + 1)
    
    return filho

## Algoritmo de Busca em largura

In [12]:
""" Definição do Nó Raiz """

#entrada da linha e coluna para a posição inicial do robo
linha = 1
coluna = 1


#se a posição inicial existir declara o estado inicial
if estado(linha,coluna):
    
    estado_inicial = [linha-1, coluna-1]
    no_inicial = No(estado_inicial, None, None, 0)
        
else:
    print("Erro")

In [58]:
""" Busca em largura """

no = no_inicial

borda = deque([no])
explorado = deque()
limite=0

while True:
    
    limite+=1
    
    if limite == 1000:
        print("Atingiu o limite em: ", no)
        solucao = avaliar_solucao(no)
        print (solucao)
        break
        
    if testar_objetivo(no):
        solucao = avaliar_solucao(no)
        print (solucao)
        break
        
    elif len(borda) == 0: 
        print('Falha ao explorar os nós')
        break
        
    else:        
        no = borda.popleft()
           
        acoes = acoes_possiveis(no.estado)

        for i, valor in enumerate(acoes):
            acao_atual = [0,0,0,0,0]
            if valor == 1:
                acao_atual[i] = 1
                filho = no_filho(no, acao_atual)  
                if len(list(filter(lambda x: x.estado == filho.estado, borda))) == 0:    
                    if not filho.acao == "Limpar":
                        borda.append(filho)    
                    elif not filho.estado in explorado:
                        borda.append(filho) 
            
                
        if no.acao == "Limpar":
            explorado.append(no.estado)
        
        # print("explorado: %s \n\n\n borda: %s \n\n\n no_atual: %s" %(explorado, borda, no))

Atingiu o limite em:  No{
 Estado: [2, 0]
 Pai: [1, 0],
 Ação: Baixo,
 Custo: 149 
}
['Baixo', 'Esquerda', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', 'Cima', 'Direita', 'Baixo', 'Esquerda', '

## Algoritmo de Busca em Profundidade


In [ ]:
no_inicial = None

""" Definição do Nó Raiz """

linha = 2
coluna = 2

limite = (18)

if estado(linha,coluna):
    estado_inicial = [linha-1, coluna-1]
    no_inicial = No(estado_inicial, None, None, 0, matriz_estados)

else:
    print("Erro")


In [ ]:
def bpl_recursiva(no_atual, limite):
    
    if testar_objetivo(no_atual.matriz_transicao):
        return avaliar_solucao(no_atual)
        
    elif limite == 1:
        return "cutoff"
    
    else:
        corte = False        
        acoes = acoes_possiveis(no_atual.estado, no_atual.matriz_transicao)
        
        if acoes == [0,0,0,0,0]:
                return "erro"
        
        else:
            for i, valor in enumerate(acoes):
                acao_atual = [0,0,0,0,0]

                if valor == 1:
                    acao_atual[i] = 1
                    filho = no_filho(no_atual, acao_atual, no_atual.matriz_transicao.copy())                
                                        
                    resultado = bpl_recursiva(filho, (limite-1))

                    if resultado == "cutoff":
                        corte = True
                            
                    elif not resultado == "erro": 
                        return resultado
               
        if corte:        
            return "cutoff"
        else:
            return "erro"
    

In [ ]:
# chama método recursivo para algoritmo de busca em profundidade

bpl_recursiva(no_inicial, limite)
print("\n\n", matriz_final)